In [3]:
import numpy as np
import pandas as pd
import multiprocessing

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline

%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

Loading BokehJS ...

In [ ]:
# SS20 = (pd.read_excel('SS20Range.xlsx', skiprows = 6).
#        drop(['Unnamed: 0', 'vlookup'], axis = 1))

# SS20_all_articles = SS20['Article Number'].unique()

# SS20 = SS20[SS20['CO'] == 'Y']
# dat['CO'].fillna('N', inplace = True)
# dat['SS19 Ranged'].fillna('N', inplace = True)
# dat['FW19 Ranged'].fillna('N', inplace = True)

# dat.to_csv('dat_SS20_range.csv')

dat_SS20_range = pd.read_csv('dat_SS20_range.csv') # *** DATA ***
SS20_range = dat_SS20_range['Article Number'].unique() # 1340

## Overstock-SupplyChain-Understock (OSU) Correction

In [ ]:
# dat = pd.read_csv('Transaction data.csv', low_memory=False) # *** DATA ***
# dat['consumer_order_date'] = pd.to_datetime(dat0['consumer_order_date'])

# # Transaction subset: SS19
# dat = dat[['consumer_order_date', 'article_number', 'gross_demand_quantity', 'Clearance', 'season']]
# dat = dat[dat['season'] == 'SS19']

# # Subset to non-clearance transactions only --- change here for David/Mike's request
# dat = dat[dat['Clearance'] == 0]

# # 'aggregate' to weekly sums by article for buy_availability merge and adjustment
# dat.set_index('consumer_order_date', inplace = True)

# dat = dat[['article_number', 'gross_demand_quantity']].groupby(['article_number']).resample('W').sum()
# dat.reset_index(inplace=True)

# # Add 'week' and 'year' for merging with stock (buy_availability) data (b/c min_date_of_week)
# dat['week'] = [t.week for t in dat['consumer_order_date']]
# dat['year'] = [t.year for t in dat['consumer_order_date']]

# dat_SS20_range = pd.read_csv('dat_SS20_range.csv') # *** DATA ***
# SS20_range = dat_SS20_range['Article Number'].unique()

# dat = dat[[(a in SS20_range) for a in dat['article_number']]]

In [57]:
# dat.to_csv('dat_SS20range.csv')

dat = pd.read_csv('dat_SS20range.csv', low_memory=False, index_col = 0) # *** DATA ***
dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])

In [58]:
# Stock data
stock = pd.read_csv('stock data.csv', low_memory=False, index_col = 0) # *** DATA ***

In [59]:
stock['year'] = [str(x)[0:4] for x in stock['week_id']]
stock['week'] = [str(x)[4:6] for x in stock['week_id']]
stock.drop('week_id', axis = 1, inplace = True)

# Tidy
stock.reset_index(inplace = True)
stock.drop(['avg(ecom_available_stock)', 'avg(size_availability)'], axis = 1, inplace = True)
stock.rename(columns = {'avg(buy_availability)': 'buy_availability'}, inplace = True)

stock = stock[(stock['year'] != '2016') & (stock['year'] != '2017')]

stock[['year', 'week']] = stock[['year', 'week']].astype('int64', copy = False)

In [60]:
# Merge weekly demand df and buy_availability df -- for understock correction
dat = pd.merge(dat, stock, 
               left_on = ['article_number', 'year', 'week'], 
               right_on = ['article_number', 'year', 'week'], 
               how = 'left')

# Assume buy_availability = 1
dat.fillna(1, inplace=True)

In [61]:
# dat[[x in ['DQ2136', 'DQ3089', 'DU0367', 'DU0369', 'DU0382', 'EE9806', 'EE9809', 'G28417'] 
#      for x in dat['article_number']]] 

# ------- Joerian: buy_availability DNE for these articles, for some reason -------

In [62]:
# ----- Calculate *observed* full season gross_demand_quantity per article -----

dat_season = pd.DataFrame(dat.groupby(['article_number'])['gross_demand_quantity'].sum())
dat_season.rename(columns = {'gross_demand_quantity':'season_gross_demand_quantity'}, inplace = True)

In [63]:
# WEEKLY averages for articles when buy_availability > 0.35
dat_stocked = pd.DataFrame(
    dat[dat['buy_availability'] > 0.35].
    groupby(['article_number'])['gross_demand_quantity'].
    mean())

# These articles have zero weeks with: 
# (1) buy_availability > 0.35 
#           *AND* 
# (2) gross_demand_quantity > 0
# {'604433', '620635', 'BQ1935', 'BQ2001', 'BS0980', 'CV4000', 'CY8772', 'G27026'}

dat_stocked.rename(columns = {'gross_demand_quantity':'corrected_weekly_avg_gross_demand_quantity'}, inplace= True)

# Extend to full season (26 weeks) to estimate full season demand
dat_stocked['corrected_gross_demand_quantity'] = 26*dat_stocked['corrected_weekly_avg_gross_demand_quantity'] 

In [64]:
# ******** MERGE ********

dat_season = pd.merge(dat_season, dat_stocked, left_index=True, right_index=True, how = 'outer')
dat_season.drop('corrected_weekly_avg_gross_demand_quantity', inplace=True, axis = 1)

In [65]:
dat_season[dat_season['season_gross_demand_quantity'] == 1486.0]

,season_gross_demand_quantity,corrected_gross_demand_quantity
article_number,,
B37616,1486.0,3489.777778


In [66]:
preds = (pd.DataFrame(pd.read_csv('Buyers predictions.csv', low_memory=False, index_col = 0))
         [['season', 'ecom_marketing_forecast']].
         reset_index().
         dropna()
        )

preds = preds[preds['season'] == 'SS19']

In [67]:
preds[preds['article'] == 'B37616']

# 	article	season	ecom_marketing_forecast
#     10588	B37616	FW18	3025.0
#     10589	B37616	FW19	3850.0

,article,season,ecom_marketing_forecast


In [13]:
# dat_season[dat_season.isna().sum(axis = 1) != 0]

# These articles have zero weeks with: 
# (1) buy_availability > 0.35 
#           *AND* 
# (2) gross_demand_quantity > 0

# set(dat_season.index).symmetric_difference(set(dat_stocked.index)) 
# {'604433', '620635', 'BQ1935', 'BQ2001', 'BS0980', 'CV4000', 'CY8772', 'G27026'}

In [68]:
dat_season['corrected_gross_demand_quantity'] = np.where(
    dat_season['corrected_gross_demand_quantity'].isna(),
    dat_season['season_gross_demand_quantity'],
    dat_season['corrected_gross_demand_quantity']
)

In [69]:
# Articles for which we are missing SS19 eCom predictions (the current season -- doesn't really matter)
(set(dat_season.index).symmetric_difference(set(preds.article))).intersection(set(dat_season.index))

{'B37616',
 'B42200',
 'BQ1935',
 'BQ2001',
 'BS0980',
 'CG1508',
 'CY1880',
 'CY5799',
 'CY5801',
 'CY6025',
 'CY8770',
 'CY8772',
 'D96635',
 'DQ1506',
 'G27026',
 'S76330'}

In [70]:
dat_season.head()
preds.head()

,season_gross_demand_quantity,corrected_gross_demand_quantity
article_number,,
011040,52.0,67.6
015110,280.0,364.0
019000,693.0,900.9
019228,174.0,226.2
019310,129.0,167.7


,article,season,ecom_marketing_forecast
6,011040,SS19,600.0
14,015110,SS19,800.0
21,019000,SS19,2000.0
29,019228,SS19,500.0
36,019310,SS19,500.0


In [71]:
dat_season = pd.merge(
    dat_season, preds, 
    left_index = True,
    right_on = 'article', 
    how = 'left').round()

In [73]:
dat_season = dat_season[['article', 'season_gross_demand_quantity', 
                         'corrected_gross_demand_quantity', 'ecom_marketing_forecast']]

In [74]:
dat_season = dat_season[dat_season['season_gross_demand_quantity'] > 83]
dat_season['DAA_SS20_prediction'] = dat_season['corrected_gross_demand_quantity']*1.1

In [76]:
dat_season[dat_season.isna().sum(axis = 1) != 0]


,article,season_gross_demand_quantity,corrected_gross_demand_quantity,ecom_marketing_forecast,DAA_SS20_prediction
106896,B37616,1486.0,3490.0,NaN,3839.0
106896,CY6025,253.0,925.0,NaN,1017.5


In [79]:
# dat_season.to_csv('dat_season.csv')
dat_season.shape
dat_season

(490, 5)

,article,season_gross_demand_quantity,corrected_gross_demand_quantity,ecom_marketing_forecast,DAA_SS20_prediction
14,015110,280.0,364.0,800.0,400.4
21,019000,693.0,901.0,2000.0,991.1
29,019228,174.0,226.0,500.0,248.6
36,019310,129.0,168.0,500.0,184.8
62,033905,498.0,647.0,1500.0,711.7
104,280647,5800.0,7540.0,40000.0,8294.0
108,280648,1487.0,1933.0,20000.0,2126.3
115,288022,3626.0,4714.0,25000.0,5185.4
191,604432,343.0,446.0,650.0,490.6
220,619995,655.0,852.0,2080.0,937.2


## Plots

In [ ]:
%whos DataFrame

In [ ]:
article = 'B96578'
# article = 'DB3258'

p = dat[dat['article_number'] == article].dropna()

pivoted = p.pivot(index = 'consumer_order_date', 
                  columns = 'article_number', 
                  values = 'buy_availability')


pivoted2 = p.pivot(index = 'consumer_order_date', 
                  columns = 'article_number', 
                  values = 'gross_demand_quantity')       

In [ ]:
# plt.xlabel('log(net_sales)')
# plt.ylabel('log(sold_qty)')
# plt.colorbar()
# plt.title('DBSCAN Classification')
# plt.rcParams["figure.figsize"] = [16,16]

In [ ]:
# plt.subplot(2, 1, 1)
# plt.scatter(dat_scaled['gross_demand_quantity']['mean'], 
#             dat_scaled['sales_price']['mean'], 
#             c = agglom,
#             alpha = 0.25)
# plt.title('Agglomerative Classes: Sale Price vs. Gross Demand Qty (mean/sum)')
# plt.ylabel('Sale Price')

# plt.subplot(2, 1, 2)
# plt.scatter(dat_scaled['gross_demand_quantity']['sum'], 
#             dat_scaled['sales_price']['mean'], 
#             c = agglom,
#             alpha = 0.25)
# plt.ylabel('Sale Price')
# plt.xlabel('GDQ Sum')

In [ ]:
plt.rcParams["figure.figsize"] = [16,5]

pivoted.plot(linewidth = 4)
plt.title('Buy Availability Over Time')
plt.ylabel('Buy Availability')

pivoted2.plot(linewidth = 4)
plt.title('Weekly Gross Demand Quantity')
plt.ylabel('Gross Demand Quantity')


### SS20 buy numbers

In [ ]:
# Wall time: 55.8 s

# SS20 buyer predictions
# SS20_buys = pd.read_excel('SS20 RMA2_May7.xlsx', skiprows = 2, sheet_name= '03 May download')
# SS20_buys = SS20_buys[['Article Number', 'Model', 'Market', 'WE eCom', 
#                        'Sub-Brand', 'Product Division', 'Marketing Segment']]


# SS20_buys.rename(columns = {'WE eCom': 'eCom SS20 Forecast'}, inplace = True)
# SS20_buys = SS20_buys[[(a in AOIs) for a in SS20_buys['Article Number']]]
# SS20_buys.to_csv('SS20_buys_carryovers.csv')

In [ ]:
dat_season = pd.read_csv('dat_season.csv', low_memory=False, index_col = 0)
SS20_buys = pd.read_csv('SS20_buys_carryovers.csv', low_memory=False, index_col = 0)

In [ ]:
# ******** MERGE ********

dat_season = (pd.merge(dat_season, SS20_buys[['Article Number', 'eCom SS20 Forecast']], 
                       left_on='article', right_on='Article Number', how = 'outer'))

dat_season = dat_season[['article', 'Article Number', 'season_gross_demand_quantity', 
                         'corrected_gross_demand_quantity', 'ecom_marketing_forecast',
                         'DAA_SS20_prediction', 'eCom SS20 Forecast']]

dat_season['prediction_difference'] = dat_season['eCom SS20 Forecast'] - dat_season['DAA_SS20_prediction']
dat_season['abs_prediction_difference'] = abs(dat_season['eCom SS20 Forecast'] - dat_season['DAA_SS20_prediction'])

### Buyer version 

In [ ]:
dat_season.drop('article', inplace=True, axis = 1)

In [ ]:
dat_season.rename(columns = {'season_gross_demand_quantity': 'SS19 Demand',
                            'corrected_gross_demand_quantity': 'SS19 Corrected Demand',
                            'ecom_marketing_forecast': 'SS19 eCom Forecast',
                            'DAA_SS20_prediction': 'Analytics SS20 Forecast',
                            }, inplace = True)

In [ ]:
dat_season = dat_season[['Article Number', 'SS19 eCom Forecast', 'SS19 Demand', 'SS19 Corrected Demand',
       'eCom SS20 Forecast', 'Analytics SS20 Forecast', 
       'prediction_difference']]

In [ ]:
buyers = pd.read_csv('article_managers.csv', low_memory=False, index_col = 0)

In [ ]:
# ******** MERGE ********

forecasts = (pd.merge(dat_season, buyers, left_on = 'Article Number', right_index = True, how = 'outer').
             set_index('Article Number').
            rename(columns = {'CM': 'Manager'}).
            drop('prediction_difference', axis = 1))

In [ ]:
%whos DataFrame

In [ ]:
# ---------------------------------------------------------------------------------

In [ ]:
test = dat0.copy()

In [ ]:
test.head()

In [ ]:
test19 = pd.DataFrame(test[test['season'] == 'SS19'].groupby('article_number')['gross_demand_quantity'].sum())

In [ ]:
test19[test19['gross_demand_quantity'] == 1486]

In [ ]:
# ---------------------------------------------------------------------------------

In [ ]:
# missing_managers = forecasts[forecasts['Manager'].isna()]
# missing_managers.to_csv('missing_managers.csv')

# forecasts.to_csv('forecasts.csv')

### Add cost/price

In [ ]:
# ---- Add cost to DF ----
cost_price = pd.read_csv('Cost Price.csv', low_memory=False, index_col = 0)

# cost_price = cost_price[[(a in SS20_range) for a in cost_price.index]]
cost_price.rename(columns = {'avg(cost_of_sales)': 'cost'}, inplace = True)
cost_price = pd.DataFrame(cost_price['cost'].groupby(cost_price.index).mean()).round()

In [ ]:
# ******** MERGE ********

forecasts = pd.merge(forecasts, cost_price, left_index=True, right_index = True, how = 'left')

In [ ]:
forecasts.shape
forecasts.head()

In [ ]:
# Add 'avg_price' to table
# (1) Filter to the 491 SS20 articles of interest
# (2) sale price on THAT transaction: gross_sales_gross_disc_net_ret/net_qty
# (3) find average of that per article
# (4) add that to table

In [ ]:
%whos DataFrame

In [ ]:
# ---- Add price to DF ----
dat0 = pd.read_csv('Transaction data.csv', low_memory=False) # *** DATA ***
dat0['consumer_order_date'] = pd.to_datetime(dat0['consumer_order_date'])

In [ ]:
price = dat0.copy()

In [ ]:
# Wall time: 2min 8s

the490 = forecasts.index

In [ ]:
price490 = price[[(a in the490) for a in price['article_number']]]

In [ ]:
price490.loc[:,'sale_price'] = price490['net_sales']/price490['gross_demand_quantity']

In [ ]:
price490 = pd.DataFrame(price490.groupby('article_number')['sale_price'].mean())

In [ ]:
forecasts = pd.merge(forecasts, price490, left_index = True, right_index = True, how = 'left').round()

In [ ]:
forecasts['prediction_difference'] = forecasts['eCom SS20 Forecast'] - forecasts['Analytics SS20 Forecast']


In [ ]:
    forecasts['loss'] = np.where(
        forecasts['prediction_difference'] > 0, 
        forecasts['prediction_difference']*forecasts['cost'],
        forecasts['prediction_difference']*(forecasts['sale_price'] - forecasts['cost'])
    )

In [ ]:
forecasts[(forecasts.index == 'B37616') | (forecasts['SS19 Demand'] == 1486)]

In [ ]:
# Way under: everything
# Way over: DB3258, DP2398, EF2803, DQ3319